In [ ]:
import os
import yfinance as yf
import pandas as pd
from datetime import datetime
from dotenv import load_dotenv

# LangChain imports
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_ollama import ChatOllama  
from langchain_community.vectorstores import FAISS
from langchain_community.docstore.document import Document
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableSequence


In [ ]:
# -------------------------------------------------------------------
# 1. Setup
# -------------------------------------------------------------------
load_dotenv()
os.makedirs("rag_store", exist_ok=True)


In [ ]:
# -------------------------------------------------------------------
# 2. Fetch news for a ticker
# -------------------------------------------------------------------
def fetch_news_for_ticker(ticker: str, max_articles: int = 50) -> pd.DataFrame:
    tk = yf.Ticker(ticker)
    raw_news = tk.news or []
    articles = []
    for item in raw_news[:max_articles]:
        articles.append({
            "title": item.get("title", ""),
            "summary": item.get("summary", ""),
            "link": item.get("link", ""),
            "date": datetime.fromtimestamp(item.get("providerPublishTime", 0))
        })
    df = pd.DataFrame(articles)
    print(f"✅ Fetched {len(df)} news items for {ticker}")
    return df

In [ ]:

# -------------------------------------------------------------------
# 3. Build text docs from the news DataFrame
# -------------------------------------------------------------------
def build_documents_from_news(df: pd.DataFrame):
    docs = []
    for _, row in df.iterrows():
        content = f"Title: {row['title']}\nSummary: {row['summary']}\nDate: {row['date'].strftime('%Y-%m-%d')}"
        docs.append(content)
    return docs


In [ ]:


# -------------------------------------------------------------------
# 4. Create (or update) the RAG vector store
# -------------------------------------------------------------------
def create_or_update_rag_store(ticker: str, docs):
    ticker_clean = ticker.replace("^", "").replace("=", "")
    save_path = f"rag_store/faiss_{ticker_clean}"
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

    documents = [Document(page_content=doc, metadata={"ticker": ticker}) for doc in docs]

    if os.path.exists(save_path):
        # incremental update
        vectorstore = FAISS.load_local(save_path, embeddings, allow_dangerous_deserialization=True)
        vectorstore.add_documents(documents)
        print(f"🧩 Added {len(documents)} docs to existing store for {ticker}")
    else:
        vectorstore = FAISS.from_documents(documents, embedding=embeddings)
        print(f"📦 Created new store for {ticker} with {len(documents)} docs")

    vectorstore.save_local(save_path)
    return vectorstore



In [ ]:
# -------------------------------------------------------------------
# 5. Query the RAG store
# -------------------------------------------------------------------
def query_rag_store(ticker: str, query: str, k: int = 5):
    ticker_clean = ticker.replace("^", "").replace("=", "")
    load_path = f"rag_store/faiss_{ticker_clean}"
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

    if not os.path.exists(load_path):
        raise FileNotFoundError(f"No RAG store found for {ticker} at {load_path}")

    vectorstore = FAISS.load_local(load_path, embeddings, allow_dangerous_deserialization=True)
    results = vectorstore.similarity_search(query, k=k)
    return results




In [ ]:
# -------------------------------------------------------------------
# 6. Generate summary / reasoning using the retrieved docs
# -------------------------------------------------------------------
def generate_reasoning_from_rag(ticker: str, query: str):
    docs = query_rag_store(ticker, query)
    context = "\n\n".join([d.page_content for d in docs])

    llm = ChatOllama(model="mistral")
    print(llm.invoke("Hello"))
    prompt = PromptTemplate(
        input_variables=["context", "query"],
        template=(
            "You are a market analyst.\n"
            "Using only the recent news below, answer the question clearly and concisely.\n\n"
            "Recent News:\n{context}\n\n"
            "Question:\n{query}\n\n"
            "Answer:"
        )
    )
    chain = RunnableSequence(first=prompt, last=llm)
    answer = chain.invoke({"context": context, "query": query})
    
    return answer


In [ ]:
# -------------------------------------------------------------------
# 7. Driver: build RAG for multiple tickers & test summary
# -------------------------------------------------------------------
tickers = ["^GSPC", "SPY", "BTC-USD", "ETH-USD"]  

for t in tickers:
    df = fetch_news_for_ticker(t)
    if df.empty:
        print(f"No news for {t}")
        continue
    docs = build_documents_from_news(df)
    create_or_update_rag_store(t, docs)

# Example: generate a reasoning summary for S&P 500
query = "What factors have recently influenced S&P 500 performance?"
response = generate_reasoning_from_rag("^GSPC", query)
print("\n🧠 Explanation:\n", response)


✅ Fetched 2 news items for ^GSPC


C:\Users\haifa\AppData\Local\Temp\ipykernel_16520\3970462769.py:7: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


🧩 Added 2 docs to existing store for ^GSPC
✅ Fetched 0 news items for SPY
No news for SPY
✅ Fetched 10 news items for BTC-USD
🧩 Added 10 docs to existing store for BTC-USD
✅ Fetched 0 news items for ETH-USD
No news for ETH-USD


ResponseError: model requires more system memory (3.5 GiB) than is available (2.3 GiB) (status code: 500)